In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wixarika-spanish/train.es
/kaggle/input/wixarika-spanish/train.hch
/kaggle/input/wixarika-spanish/dev.es
/kaggle/input/wixarika-spanish/test.hch-es.src
/kaggle/input/wixarika-spanish/test.es-hch.src
/kaggle/input/wixarika-spanish/dev.hch


In [2]:
%%capture
%pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu -q

Import libraries

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, NllbTokenizer
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
from sklearn.model_selection import train_test_split
import re
import random
from tqdm.auto import tqdm
import os

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `basic task` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `basic task`


## Dataset

In [5]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/wixarika-spanish/train.hch', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/wixarika-spanish/train.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
train_df  = pd.DataFrame({
    # 'Source': source,
    'Wixarika': ashaninka_lines,
    'Spanish': spanish_lines
})

In [6]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/wixarika-spanish/dev.hch', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/wixarika-spanish/dev.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
dev_df  = pd.DataFrame({
    # 'Source': source,
    'Wixarika': ashaninka_lines,
    'Spanish': spanish_lines
})

In [7]:
# Create a test set from the dev data
dev_df, test_df = train_test_split(dev_df, test_size=0.5, random_state=42)

In [8]:
print(f"Training samples: {len(train_df)}")
print(f"Development samples: {len(dev_df)}")
print(f"Test samples: {len(test_df)}")

Training samples: 8966
Development samples: 497
Test samples: 497


In [9]:
# Step 4: Initialize the NLLB tokenizer and model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = NllbTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Examine tokenization

In [10]:
def word_tokenize(text):
    return re.findall('(\w+|[^\w\s])', text)

In [11]:
# Sample from the training data
sample = train_df.sample(min(10000, len(train_df)), random_state=1)

In [12]:
# Apply tokenization
sample['hch_toks'] = sample.Wixarika.apply(tokenizer.tokenize)
sample['spa_toks'] = sample.Spanish.apply(tokenizer.tokenize)
sample['hch_words'] = sample.Wixarika.apply(word_tokenize)
sample['spa_words'] = sample.Spanish.apply(word_tokenize)

In [13]:
# Compute statistics
stats = sample[['hch_toks', 'spa_toks', 'hch_words', 'spa_words']].applymap(len).describe()

<ipython-input-13-eb2b22f5ac33>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stats = sample[['hch_toks', 'spa_toks', 'hch_words', 'spa_words']].applymap(len).describe()


In [14]:
print("Average tokens per word for Wixarika:", stats.hch_toks['mean'] / stats.hch_toks['mean'])
print("Average tokens per word for Spanish:", stats.spa_toks['mean'] / stats.spa_words['mean'])

Average tokens per word for Wixarika: 1.0
Average tokens per word for Spanish: 1.2073756088356349


In [15]:
# Check for unknown tokens
texts_with_unk_hch = [
    text for text in tqdm(train_df.Wixarika)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

texts_with_unk_spa = [
    text for text in tqdm(train_df.Spanish)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

  0%|          | 0/8966 [00:00<?, ?it/s]

  0%|          | 0/8966 [00:00<?, ?it/s]

In [16]:
print(f"Number of Wixarika texts with unknown tokens: {len(texts_with_unk_hch)}")
print(f"Number of Spanish texts with unknown tokens: {len(texts_with_unk_spa)}")

Number of Wixarika texts with unknown tokens: 4
Number of Spanish texts with unknown tokens: 1


## Add Language Token if needed

In [17]:
def fix_tokenizer(tokenizer, new_lang='hch_Latn'):
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset
    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [18]:
if 'agr_Latn' not in tokenizer.lang_code_to_id:
    fix_tokenizer(tokenizer, 'hch_Latn')
    model.resize_token_embeddings(len(tokenizer))
    
    # Initialize the embedding for the new token with a similar language
    added_token_id = tokenizer.convert_tokens_to_ids('hch_Latn')
    similar_lang_id = tokenizer.convert_tokens_to_ids('quy_Latn')  # Quechua is similar to Aymara
    embeds = model.model.shared.weight.data
    embeds[added_token_id] = embeds[similar_lang_id]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


## Preprocess function for text

In [19]:
def preproc(text):
    from sacremoses import MosesPunctNormalizer
    mpn = MosesPunctNormalizer(lang="en")
    clean = mpn.normalize(text)
    return clean

## Training function

In [20]:
def get_batch_pairs(batch_size, data=train_df):
    LANGS = [('Wixarika', 'hch_Latn'), ('Spanish', 'spa_Latn')]
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

In [21]:
def train_model(model, tokenizer, training_steps=20000, batch_size=16, max_length=128, lr=1e-4):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Define optimizer and scheduler
    optimizer = Adafactor(
        [p for p in model.parameters() if p.requires_grad],
        scale_parameter=False,
        relative_step=False,
        lr=lr,
        clip_threshold=1.0,
        weight_decay=1e-3,
    )
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)
    
    # Training loop
    losses = []
    MODEL_SAVE_PATH = 'nllb-aym-spa-v1'
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    model.train()
    x, y, loss = None, None, None
    
    def cleanup():
        import gc
        gc.collect()
        torch.cuda.empty_cache()
    
    for i in tqdm(range(training_steps)):
        xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
        
        try:
            tokenizer.src_lang = lang1
            x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            tokenizer.src_lang = lang2
            y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100
            
            loss = model(**x, labels=y.input_ids).loss
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            
        except RuntimeError as e:
            optimizer.zero_grad(set_to_none=True)
            x, y, loss = None, None, None
            cleanup()
            print('error', max(len(s) for s in xx + yy), e)
            continue
        
        if i % 1000 == 0:
            avg_loss = torch.mean(torch.tensor(losses[-1000:] if len(losses) > 1000 else losses))
            print(f"Step {i}, Average Loss: {avg_loss}")
            if i % 5000 == 0 and i > 0:
                model.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
                tokenizer.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
    
    # Save the final model
    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)
    
    return model, tokenizer

## Evaluation function

In [22]:
def evaluate_model(model, tokenizer, test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Function to translate text
    def translate(text, src_lang, tgt_lang, num_beams=4):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate Aymara to Spanish
    hch_to_spa = []
    for text in tqdm(test_data.Wixarika):
        hch_to_spa.append(translate(text, 'hch_Latn', 'spa_Latn')[0])
    
    # Translate Spanish to Aymara
    spa_to_hch = []
    for text in tqdm(test_data.Spanish):
        spa_to_hch.append(translate(text, 'spa_Latn', 'hch_Latn')[0])
    
    # Calculate BLEU scores
    import sacrebleu
    
    bleu_hch_spa = sacrebleu.corpus_bleu(hch_to_spa, [test_data.Spanish.tolist()])
    bleu_spa_hch = sacrebleu.corpus_bleu(spa_to_hch, [test_data.Wixarika.tolist()])
    
    chrf_hch_spa = sacrebleu.corpus_chrf(hch_to_spa, [test_data.Spanish.tolist()])
    chrf_spa_hch = sacrebleu.corpus_chrf(spa_to_hch, [test_data.Wixarika.tolist()])
    
    # Calculate exact match accuracy
    hch_spa_exact_matches = sum(1 for pred, ref in zip(hch_to_spa, test_data.Spanish) if pred.strip() == ref.strip())
    spa_hch_exact_matches = sum(1 for pred, ref in zip(spa_to_hch, test_data.Wixarika) if pred.strip() == ref.strip())
    
    hch_spa_accuracy = (hch_spa_exact_matches / len(test_data)) * 100
    spa_hch_accuracy = (spa_hch_exact_matches / len(test_data)) * 100
    
    # Calculate word-level accuracy (percentage of words that match)
    def calculate_word_accuracy(predictions, references):
        total_words = 0
        correct_words = 0
        
        for pred, ref in zip(predictions, references):
            pred_words = pred.strip().split()
            ref_words = ref.strip().split()
            
            # Count words in reference
            total_words += len(ref_words)
            
            # Count matching words (position-sensitive)
            for i, ref_word in enumerate(ref_words):
                if i < len(pred_words) and pred_words[i] == ref_word:
                    correct_words += 1
        
        return (correct_words / total_words) * 100 if total_words > 0 else 0
    
    hch_spa_word_accuracy = calculate_word_accuracy(hch_to_spa, test_data.Spanish)
    spa_hch_word_accuracy = calculate_word_accuracy(spa_to_hch, test_data.Wixarika)
    
    print(f"BLEU Wixarika → Spanish: {bleu_hch_spa.score:.2f}")
    print(f"BLEU Spanish → Wixarika: {bleu_spa_hch.score:.2f}")
    print(f"chrF++ Wixarika → Spanish: {chrf_hch_spa.score:.2f}")
    print(f"chrF++ Spanish → Wixarika: {chrf_spa_hch.score:.2f}")
    print(f"Exact Match Accuracy Wixarika → Spanish: {hch_spa_accuracy:.2f}%")
    print(f"Exact Match Accuracy Spanish → Wixarika: {spa_hch_accuracy:.2f}%")
    print(f"Word-level Accuracy Wixarika → Spanish: {hch_spa_word_accuracy:.2f}%")
    print(f"Word-level Accuracy Spanish → Wixarika: {spa_hch_word_accuracy:.2f}%")
    
    return {
        'bleu_hch_spa': bleu_hch_spa.score,
        'bleu_spa_hch': bleu_spa_hch.score,
        'chrf_hch_spa': chrf_hch_spa.score,
        'chrf_spa_hch': chrf_spa_hch.score,
        'hch_spa_accuracy': hch_spa_accuracy,
        'spa_hch_accuracy': spa_hch_accuracy,
        'hch_spa_word_accuracy': hch_spa_word_accuracy,
        'spa_hch_word_accuracy': spa_hch_word_accuracy,
        'hch_to_spa_samples': list(zip(test_data.Wixarika.head(5), hch_to_spa[:5])),
        'spa_to_hch_samples': list(zip(test_data.Spanish.head(5), spa_to_hch[:5]))
    }

## Main Execution

In [23]:
# Train the model
print("Starting model training...")
model, tokenizer = train_model(model, tokenizer, training_steps=20000) #Min 10000 steps need to be a good translator

Starting model training...


  0%|          | 0/20000 [00:00<?, ?it/s]

Step 0, Average Loss: 8.180397033691406
Step 1000, Average Loss: 3.556321144104004
Step 2000, Average Loss: 2.2311153411865234
Step 3000, Average Loss: 1.7652240991592407
Step 4000, Average Loss: 1.4463536739349365
Step 5000, Average Loss: 1.1966402530670166
Step 6000, Average Loss: 0.9992607235908508
Step 7000, Average Loss: 0.8370302319526672
Step 8000, Average Loss: 0.6896238923072815
Step 9000, Average Loss: 0.5702376961708069
Step 10000, Average Loss: 0.4673170745372772
Step 11000, Average Loss: 0.3854667842388153
Step 12000, Average Loss: 0.3148352801799774
Step 13000, Average Loss: 0.2629278600215912
Step 14000, Average Loss: 0.21449993550777435
Step 15000, Average Loss: 0.1753523349761963
Step 16000, Average Loss: 0.15315234661102295
Step 17000, Average Loss: 0.13186389207839966
Step 18000, Average Loss: 0.11591430008411407
Step 19000, Average Loss: 0.10176052898168564


## Hugging Face Model Push

In [24]:
model_name = "mahsharyahan/nllb-600B-hch-spa-v1"
tokenizer.push_to_hub(model_name)
model.push_to_hub(model_name)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahsharyahan/nllb-600B-hch-spa-v1/commit/cdb1d2287b704d754ac6f970701e3a29f5f61bc0', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='cdb1d2287b704d754ac6f970701e3a29f5f61bc0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mahsharyahan/nllb-600B-hch-spa-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='mahsharyahan/nllb-600B-hch-spa-v1'), pr_revision=None, pr_num=None)

In [25]:
# Evaluate the model
print("Evaluating the model...")
eval_results = evaluate_model(model, tokenizer, test_df)

Evaluating the model...


  0%|          | 0/497 [00:00<?, ?it/s]

  0%|          | 0/497 [00:00<?, ?it/s]

BLEU Wixarika → Spanish: 3.69
BLEU Spanish → Wixarika: 8.66
chrF++ Wixarika → Spanish: 23.36
chrF++ Spanish → Wixarika: 29.11
Exact Match Accuracy Wixarika → Spanish: 0.00%
Exact Match Accuracy Spanish → Wixarika: 0.00%
Word-level Accuracy Wixarika → Spanish: 3.63%
Word-level Accuracy Spanish → Wixarika: 2.84%


## Sample Testing

In [26]:
# Print some translation examples
print("\Awajun to Spanish translation examples:")
for src, tgt in eval_results['hch_to_spa_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

print("\nSpanish to Awajun translation examples:")
for src, tgt in eval_results['spa_to_hch_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

\Awajun to Spanish translation examples:
Source: Becky p+ h+k+ m+tiyu +kit+a ik+ 8° grado.
Translation: -"Está pegada en esta capa para el octo grado.

Source: y+ wa+kawa me+y+n+ ´+.
Translation: en mucho a la larga.

Source: ik+ presidente meny+k+ xexe+riwa mentewa ku xaxat+wa ik+ administracion.
Translation: este presidente se ha jactado de que ha dicho esto a la Administración.

Source: ipa+ pey+ kepa+ men netse yuri ne mi hayewa k+.
Translation: éste es el camino que conduciré hacia donde el hombre que me despreció.

Source: mepeta xei ik+ kí meta waniu x+nari kí wewi wame,tsepa waniu tita waniu muwa memeti yaxek+ pewekai yeme h+xiena wana Broad Street.
Translation: encontraron esta casa y la encendieron una pieza más grande, que estaba justamente enfrente de la calle.


Spanish to Awajun translation examples:
Source: Becky es una estudiante de 8° grado.
Translation: becky p+h+k+ x+ka de ataxewienieri.

Source: Está compuesto por voluntarios.
Translation: mep+y+we yu voluntari.

So

## Submit Function

In [27]:
def test_translation_without_targets(model, tokenizer, test_data, src_lang, tgt_lang, num_samples=5):
    import torch
    from tqdm.auto import tqdm
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Determine column name based on language direction
    if src_lang == 'hch_Latn':
        src_col = 'Wixarika'
    else:
        src_col = 'Spanish'
    
    # Function to translate text
    def translate(text, num_beams=4, max_length=128):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams,
                max_length=max_length
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate test samples
    translations = []
    
    # Select random samples for detailed review
    test_samples = test_data.sample(min(num_samples, len(test_data)))
    
    # Process all test data
    print(f"\n===== Translation Results: {src_lang} to {tgt_lang} =====")
    print("Sample Translations:")
    
    for i, (_, row) in enumerate(tqdm(test_data.iterrows(), total=len(test_data), desc=f"Translating {src_lang} to {tgt_lang}")):
        source_text = row[src_col]
        translated_text = translate(source_text)[0]
        translations.append(translated_text)
        
        # Display detailed examples for the selected samples
        if _ in test_samples.index:
            print(f"\nExample {i+1}:")
            print(f"Source ({src_col}): {source_text}")
            print(f"Translation: {translated_text}")
    
    return translations

In [28]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/wixarika-spanish/test.hch-es.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_hch_es = pd.DataFrame({
    # 'Source': source,
    'Wixarika': ashaninka_lines,
})

In [29]:
hch_to_es_results = test_translation_without_targets(model, tokenizer, df_test_hch_es, 'hch_Latn', 'spa_Latn')


===== Translation Results: hch_Latn to spa_Latn =====
Sample Translations:


Translating hch_Latn to spa_Latn:   0%|          | 0/1003 [00:00<?, ?it/s]


Example 83:
Source (Wixarika): tapini kí pewekai ik+ hutame hakuye yeukaku.
Translation: nuestra casa estaba bajo este y esta mitad del río.

Example 134:
Source (Wixarika): hakewa meuyakai ni xewit+ kwara maikai.
Translation: donde nadie me había conocido.

Example 215:
Source (Wixarika): ne mama putiuta uximaya tax+ atewa.
Translation: y mi madre trabaja como sirviente.

Example 548:
Source (Wixarika): mepeti tekai iyarieya tsie ik+ Augusta.
Translation: y sentaron en el corazón de esta flor flor.

Example 886:
Source (Wixarika): painekai kename neka iyari xiekakie kai.
Translation: y pensó que no se había sentido forzado a disparar su enojo.


In [30]:
# Create a DataFrame
df_test_result_hch_to_es = pd.DataFrame({
    # 'Source': source,
    'Spanish': hch_to_es_results,
})

In [31]:
df_test_result_hch_to_es.to_csv('test_result_hch_to_es.tsv', sep='\t', index=False, header=False)

In [32]:
from IPython.display import FileLink

FileLink("test_result_hch_to_es.tsv")

/kaggle/working/test_result_hch_to_es.tsv

In [33]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/wixarika-spanish/test.es-hch.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_es_hch = pd.DataFrame({
    # 'Source': source,
    'Spanish': ashaninka_lines,
})

In [34]:
es_to_hch_results = test_translation_without_targets(model, tokenizer, df_test_es_hch, 'spa_Latn','hch_Latn')


===== Translation Results: spa_Latn to hch_Latn =====
Sample Translations:


Translating spa_Latn to hch_Latn:   0%|          | 0/1003 [00:00<?, ?it/s]


Example 8:
Source (Spanish): Hubo un tiroteo cerca de mi casa, realmente no es bueno en esta área.
Translation: puye tuaxi hulawa ne kie, deyuri 'aix+ kati'aneme ena.

Example 37:
Source (Spanish): Tu tiempo nunca supera los tres días.
Translation: a tukari kapariewi ni heiwamie haika tukari.

Example 103:
Source (Spanish): Así que vivimos en esta zona.
Translation: m+pa+ri teukakuteniri ena.

Example 435:
Source (Spanish): Quiero decir que ellos solo tenían, algo así como, cinco hijos. Uno de ellos murió.
Translation: ne neuxeiya kename m+k+ xeik+a mepexeiyakai, tsipem+k+ yu niwema yu xewit+ pem+k+.

Example 845:
Source (Spanish): ¡Vaya! cuando tenía un Cocker Spaniel, era un perro en el aire libre y creo que me gustaba más ¡vaya!
Translation: ¡waya! kepa+ka mexeiyakai xeime cocker spaniel, peh+k+t+kai ts+k+ wapait+ neputi nakekai matsi wa+ka ¡waya!


In [35]:
# Create a DataFrame
df_test_result_es_to_hch = pd.DataFrame({
    # 'Source': source,
    'Awajun': es_to_hch_results,
})

In [36]:
df_test_result_es_to_hch.to_csv('test_result_es_to_hch.tsv', sep='\t', index=False, header=False)

In [37]:
from IPython.display import FileLink

FileLink("test_result_es_to_hch.tsv")

/kaggle/working/test_result_es_to_hch.tsv